In [ ]:
!pip install langchain_experimental pandas  scikit-learn langchain_tavily matplotlib

In [ ]:
from dotenv import load_dotenv
load_dotenv()


In [4]:
from langchain.agents import tool
from langchain.embeddings import OpenAIEmbeddings
 
@tool
def get_length_text(text) :
    """TEXT 의 글자수를 반환하는 함수"""
    return len(text.strip("'\n"))

@tool
def get_root(number):
    """숫자의 제곱수를 반환하는 함수"""
    n = int(number)
    return n * n

In [5]:
from langchain_core.prompts import PromptTemplate
prompt_text = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question : {input}
Thought:{agent_scratchpad}
"""

prompt = PromptTemplate.from_template(prompt_text)

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
tools = [get_length_text,get_root]
agent = create_react_agent(llm, tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input" : 'BCCARD 의 글자수의 제곱은 몇인가요'})


### Langchain Tool LIST [https://python.langchain.com/docs/integrations/tools/]

In [ ]:
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL
python_repl = PythonREPL()
python_repl.run('print(1+1)')

In [13]:
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python command",
    func=python_repl.run
)

In [ ]:
repl_tool.invoke("print(1+1)")

In [15]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
prompt = ChatPromptTemplate.from_messages([
    ('system', 
     '당신은 파이썬 프로그래밍 전문가 입니다. 파이썬 표준 문법에 따라 코드를 작성해주세요'
     '답변은 코드만 작성해주세요. no intro, no explanation, no chatty, no markdown, no code block. response only python code'),
    ('human', '{input}')
])
chain = prompt | llm | StrOutputParser()

In [ ]:
response = chain.invoke("sklearn 을 이용한 데이터 분석  예시 코드. 데이터는 인터넷에서 로드해서")
print(response)

In [ ]:
result = repl_tool.invoke(response)
print(result)

### Tavily 검색도구 [https://tavily.com/]

In [31]:
from langchain_tavily import TavilySearch
tavily_tool = TavilySearch(
    max_results=5,
    topic="general",
    # include_answer=False,
    # include_raw_content=False,
    # include_images=False,
    # include_image_descriptions=False,
    # search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)

In [ ]:
tavily_tool.invoke("아스날 파리 챔스 결과")

In [ ]:
from langchain_tavily import TavilySearch
from langgraph.prebuilt import create_react_agent

# Initialize Tavily Search Tool
tavily_search_tool = TavilySearch(
    max_results=5,
    topic="general",
)

agent = create_react_agent(llm, [repl_tool,tavily_search_tool])

user_input = "아스날 파리 챔스 결과"

for step in agent.stream(
    {"messages": user_input},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
response = agent.invoke({"messages" : "최근 한달간의 코스피 지수를 분석해서 파이썬 분석을 통해 다음달 오른다 내린다? 판단해줘"})

In [ ]:
print(response['messages'][-1].content)

In [ ]:
agent